### 1-Install and import required libraries

In [1]:
!pip install nibabel

In [167]:
import numpy as np
import nibabel as nb
import pandas as pd
from google.colab import files
import os
import csv 

### Import niftii files from github

In [71]:
!git clone https://github.com/theibsi/data_sets.git

Cloning into 'data_sets'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 11267 (delta 6), reused 40 (delta 6), pack-reused 11226
Receiving objects: 100% (11267/11267), 563.05 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (1295/1295), done.
Checking out files: 100% (10332/10332), done.


In [81]:
data_path = 'data_sets/ibsi_1_digital_phantom/nifti'
!ls
mask_data = nb.load(os.path.join(data_path, 'mask', 'mask.nii.gz')).get_fdata()
phantom_data = nb.load(os.path.join(data_path, 'image', 'phantom.nii.gz')).get_fdata()

data_sets  mask.nii.gz	phantom.nii.gz	sample_data


#### Note that it is also possible to import data from local by using files.upload as below

In [3]:
uploaded = files.upload()

Saving phantom.nii.gz to phantom.nii.gz


In [4]:
files.upload()

Saving mask.nii.gz to mask.nii.gz


{'mask.nii.gz': b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x0b\x8bad \x02\x14103\xb02\xb0\x00!#\x14\xa2\x02\x16\x06\x010\xdd`\x0f$\x1c\x900\x1a\xd8\xe0\x0cU\x03\x02L\xc4\xd8LK\xc0\x84\xe2\x0f\xb0\xbb\x1a\xa0\x18\x04\x0e K\xa2\x895 \xc99`3;O\xdb\x10L3R\x00q\xeb\xc64\x19\xbbZ\xc2.\x00\x00\xd6\x08\xe9J\x00\x02\x00\x00'}

In [5]:
mask_data = nb.load('mask.nii.gz').get_fdata()
phantom_data = nb.load('phantom.nii.gz').get_fdata()

#### some sanity checks

In [82]:
#verifying the shape of mask and phantom are similar
mask_data.shape == phantom_data.shape


True

In [84]:
# compute the phantom with the mask 
phantom_masked = phantom_data * mask_data

# checking
print(np.unique(mask_data))
print(np.unique(phantom_data))
print(np.unique(phantom_masked))

[0. 1.]
[1. 3. 4. 6. 9.]
[0. 1. 3. 4. 6.]


In [85]:
phantom_masked

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [4., 0., 1., 1.],
        [4., 4., 1., 1.]],

       [[4., 4., 4., 4.],
        [4., 1., 1., 1.],
        [1., 1., 1., 1.],
        [4., 4., 1., 1.]],

       [[4., 4., 4., 4.],
        [6., 6., 1., 1.],
        [6., 3., 0., 1.],
        [6., 6., 6., 6.]],

       [[1., 1., 0., 0.],
        [1., 1., 1., 1.],
        [4., 1., 1., 1.],
        [4., 1., 1., 1.]],

       [[1., 1., 0., 0.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

#### Compute features and store values into a dictionnary where each key correponds to the current feature name

In [148]:
features_dict = {}
# mean intensity
features_dict['mean_intensity'] = phantom_masked.flatten().mean()
print(features_dict['mean_intensity'])

1.9875


In [149]:
#calculate mean variance
features_dict['intensity_variance'] = ((phantom_masked - features_dict['mean_intensity'])**2).mean()
features_dict['intensity_variance']

3.1373437500000003

In [150]:
# Intensity skewness
num = ((phantom_data - features_dict['mean_intensity'])**2).mean()
denom = features_dict['intensity_variance']**1.5
features_dict['intensity_skewness']= num / denom 
features_dict['intensity_skewness']

0.6328408108841197

In [151]:
# intensity kurtosis
num = ((phantom_masked - mean_intensity)**4).mean()
denom = features_dict['intensity_variance']**2 
features_dict['intensity kurtosis'] = (num / denom) - 3
features_dict['intensity kurtosis'] 

-0.18175094675151637

In [152]:
#Median intensity
features_dict['median_intensity'] = np.median(phantom_masked.flatten())
features_dict['median_intensity']

1.0

In [153]:
#minimum intensity
features_dict['min_intensity'] = (phantom_masked.flatten()).min()
features_dict['min_intensity']

0.0

In [154]:
# the 10th percentile
features_dict['tenth_percentile'] = np.percentile(phantom_masked.flatten(), 10)
features_dict['tenth_percentile']

1.0

In [155]:
# the 90 percentile
features_dict['ninetenth_percentile'] = np.percentile(phantom_masked.flatten(), 90)
features_dict['ninetenth_percentile']

4.0

In [156]:
# maximum intensity
features_dict['max_intensity'] = phantom_masked.flatten().max()
features_dict['max_intensity']

6.0

In [157]:
# intensity interquartile range
features_dict['interquartile_range'] = np.percentile(phantom_masked.flatten(), 75) - np.percentile(phantom_masked.flatten(), 25)
features_dict['interquartile_range']

3.0

In [158]:
# intensity range
features_dict['intensity range'] = features_dict['max_intensity'] - features_dict['min_intensity']
features_dict['intensity range']

6.0

In [159]:
# intensity-based-mean absolute deviation
features_dict['mean_absolute_deviation'] = (np.abs(phantom_masked - features_dict['mean_intensity'])).mean()
features_dict['mean_absolute_deviation']

1.5325000000000002

In [160]:
# intensity-based robust mean absolute deviation
P10_90 = phantom_masked[np.where((features_dict['tenth_percentile'] <= phantom_masked) & (phantom_masked <= features_dict['ninetenth_percentile']))] 
features_dict['mean_absolute_deviation'] = (np.abs(P10_90 - P10_90.mean())).mean()
features_dict['mean_absolute_deviation']

1.1138338159946537

In [161]:
# intensity-based median absolute deviation
features_dict['median_absolute_deviation'] = (np.abs(phantom_masked - features_dict['median_intensity'])).mean()
features_dict['median_absolute_deviation']

1.1375

In [162]:
# intensity-based coefficient of variation
features_dict['coefficient_of_variation'] = features_dict['intensity_variance'] / features_dict['mean_intensity']
features_dict['coefficient_of_variation']

1.5785377358490567

In [163]:
# Intensity-based quartile coefficient of dispersion
P25 = np.percentile(phantom_masked, 25)
P75 = np.percentile(phantom_masked, 75)
features_dict['quartile_coefficient_of_dispersion'] = (P75 - P25) / (P75 + P25)
features_dict['quartile_coefficient_of_dispersion']

0.6

In [164]:
# intensity based energy
features_dict['energy'] = (np.square(phantom_masked)).sum()
features_dict['energy']

567.0

In [165]:
# root mean square intensity
features_dict['root_mean_square_intensity'] = np.sqrt(features_dict['energy'] / len(phantom_masked.flatten()))
features_dict['root_mean_square_intensity']

2.662235902394827

### exporting results to a CSV file and download

In [169]:
# round results with lot of decimals up to 2 decimal
print(features_dict)
features_dict = {key : round(value, 2) for key,value in features_dict.items()}
print(features_dict)

# convert the dict to a dataframe then export it to csv format
df = pd.DataFrame(list(features_dict.items()), columns=['feature', 'value'])
df.to_csv('features.csv')


{'mean_intensity': 1.99, 'intensity_variance': 3.14, 'intensity_skewness': 0.63, 'intensity kurtosis': -0.18, 'median_intensity': 1.0, 'min_intensity': 0.0, 'tenth_percentile': 1.0, 'ninetenth_percentile': 4.0, 'max_intensity': 6.0, 'interquartile_range': 3.0, 'intensity range': 6.0, 'mean_absolute_deviation': 1.11, 'median_absolute_deviation': 1.14, 'coefficient_of_variation': 1.58, 'quartile_coefficient_of_dispersion': 0.6, 'energy': 567.0, 'root_mean_square_intensity': 2.66}
{'mean_intensity': 1.99, 'intensity_variance': 3.14, 'intensity_skewness': 0.63, 'intensity kurtosis': -0.18, 'median_intensity': 1.0, 'min_intensity': 0.0, 'tenth_percentile': 1.0, 'ninetenth_percentile': 4.0, 'max_intensity': 6.0, 'interquartile_range': 3.0, 'intensity range': 6.0, 'mean_absolute_deviation': 1.11, 'median_absolute_deviation': 1.14, 'coefficient_of_variation': 1.58, 'quartile_coefficient_of_dispersion': 0.6, 'energy': 567.0, 'root_mean_square_intensity': 2.66}


In [170]:
# download the csv
files.download('features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>